## Stats507 Homework 10, May 2, 2019
### Israel Diego
#### israeldi@umich.edu

This notebook shows solution to Problem 3, homework 10 for Stats507

## Table of Contents

1. [Problem 3: Building a Complicated Model](#Problem-3:-Building-a-Complicated-Model)

### Problem 3: Building a Complicated Model
#### Time Spent: 
#### ([Back to Top](#Table-of-Contents))

The TensorFlow documentation includes tutorials on building a number of more complicated neural models in TensorFlow: https://www.tensorflow.org/tutorials/. In the left side panel, choose any one tutorial from under one of the headings "ML at production scale", "Generative models", "Images" or "Sequences" and follow it. Some of the tutorials include instructions along the lines of "We didn't discuss this trick, try adding it!". You do not need to do any of these additional steps (though you will certainly learn something if you do!). **Warning:** some of the tutorials require large amounts of training data. If this is the case, please do not include the training data in your submission! Instead, include a line of code to download the data from wherever it is stored. Also, some of the tutorials require especially long training time, (e.g., the neural models) so budget your time accordingly!

Your submission for this problem should be a *separate* jupyter notebook called `tutorial.ipynb` (no need to include your uniqname), which includes code to load the training and test data, build and train a model, and evaluate that model on test data. That is, the code in `tutorial.ipynb` should perform all the training and testing steps performed in the tutorial, but without having to be run from the command line. Depending on which model you choose, training may take a long time if you use the preset number of training steps, so be sure to include a variable called `nsteps` that controls the number of training steps, and set it to be something moderately small for your submission.

**Note:** it will not be enough to simply copy the tutorial's python code into your
jupyter notebook, since the demo code supplied in the tutorials is meant to be run from
the command line.

**Another note:** If it was not clear, you are, for this problem and this problem only,
permitted to copy-paste code from the TensorFlow tutorials as much as you like without
penalty.

**One more note:** Please make sure that in both `tutorial.ipynb` and your main submission notebook `uniqname.hw10.ipynb` you do not set any training times to be excessively long. You are free to set the number of training steps as you like for running on your own machine, but please set these parameters to something more reasonable in your submission so that we do not need to wait too long when running your notebook. Aim to set the number of training steps so that we can run each of your submitted notebooks less than a minute. 

In [1]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_data},
  y=train_labels,
  batch_size=100,
  num_epochs=None,
  shuffle=True)
mnist_classifier.train(
  input_fn=train_input_fn,
  steps=1000,
  hooks=[logging_hook])

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

/Users/israeldiego/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_ch

INFO:tensorflow:loss = 2.3084147, step = 1
INFO:tensorflow:probabilities = [[0.09130936 0.0902095  0.09854567 0.10275336 0.10416055 0.09926388
  0.10062259 0.10875795 0.10521689 0.0991602 ]
 [0.10352603 0.08814379 0.10601    0.08872351 0.10499515 0.09909759
  0.10963844 0.11275917 0.10612132 0.08098497]
 [0.09694017 0.09915512 0.10257494 0.09731591 0.09283771 0.10079278
  0.10613761 0.10634375 0.10167797 0.09622406]
 [0.0985686  0.10479892 0.09888628 0.09467955 0.10299761 0.11131835
  0.10203283 0.10391858 0.09915855 0.08364079]
 [0.10120805 0.09459337 0.09316538 0.0916088  0.09840298 0.10946508
  0.10964122 0.11231053 0.10724232 0.08236227]
 [0.09295839 0.10426626 0.10153885 0.10074852 0.09935129 0.10029729
  0.0983675  0.09854157 0.10956354 0.09436677]
 [0.10461136 0.1110911  0.08465994 0.1056347  0.10403214 0.1052998
  0.09585272 0.10437774 0.10372379 0.08071675]
 [0.08624576 0.09408287 0.10812771 0.10132071 0.10673719 0.1010447
  0.1013698  0.10605495 0.09750073 0.09751553]
 [0.093

INFO:tensorflow:global_step/sec: 6.1143
INFO:tensorflow:probabilities = [[0.1058318  0.09983793 0.10264479 0.09685817 0.09560951 0.09969047
  0.10855571 0.11595753 0.10091355 0.07410055]
 [0.09909545 0.10512151 0.09914415 0.10634065 0.09328403 0.09427971
  0.10412417 0.11332928 0.09757779 0.08770335]
 [0.08748218 0.11227356 0.09601685 0.10495044 0.10025964 0.09394071
  0.10441182 0.10395736 0.10645841 0.09024899]
 [0.10084137 0.08738346 0.10507619 0.11775116 0.08191758 0.12079108
  0.09948745 0.11009336 0.09608997 0.08056836]
 [0.09132738 0.10177633 0.09970776 0.09788419 0.09931518 0.10837041
  0.10090438 0.10785089 0.09792358 0.09493995]
 [0.09406317 0.09088031 0.09611177 0.0948188  0.1097     0.08857381
  0.10357198 0.11045725 0.11090741 0.10091543]
 [0.09995528 0.09748907 0.09849259 0.09624897 0.09436065 0.10144443
  0.10599039 0.11608499 0.100313   0.08962066]
 [0.09696662 0.08685477 0.09775301 0.09861644 0.10740609 0.10654488
  0.11042862 0.10553829 0.09424292 0.09564836]
 [0.0974

INFO:tensorflow:loss = 2.298429, step = 101 (16.358 sec)
INFO:tensorflow:probabilities = [[0.09681778 0.09763081 0.10051218 0.09520536 0.10274802 0.10399128
  0.09858964 0.11576717 0.09535923 0.09337854]
 [0.09356456 0.10208525 0.10795154 0.10059811 0.09408008 0.0883624
  0.10179421 0.112472   0.1016681  0.09742375]
 [0.10139289 0.10761674 0.09440128 0.09973143 0.10649094 0.10502335
  0.1009979  0.09172916 0.09768312 0.09493317]
 [0.09286381 0.10255811 0.1002417  0.10470019 0.10119116 0.1003418
  0.1016053  0.10731483 0.08717004 0.1020131 ]
 [0.08969168 0.09825681 0.10497699 0.1004077  0.10364121 0.10503658
  0.09957984 0.10035958 0.10319591 0.09485364]
 [0.10048825 0.10813714 0.09634866 0.10167977 0.08942403 0.10280956
  0.09958294 0.1104162  0.10777061 0.08334289]
 [0.09845938 0.09832358 0.09688438 0.10919604 0.10111438 0.1042797
  0.10314078 0.10726976 0.09888049 0.08245154]
 [0.09844973 0.10412604 0.0938013  0.10333367 0.10008144 0.09431764
  0.10951364 0.10552078 0.10437127 0.0864

...

INFO:tensorflow:global_step/sec: 6.11461
INFO:tensorflow:probabilities = [[0.10451827 0.07338697 0.0990877  0.10195677 0.11691268 0.11006392
  0.10101966 0.08435498 0.10479715 0.10390189]
 [0.2060172  0.05784605 0.10306571 0.08842734 0.08413018 0.11327603
  0.1107958  0.07517485 0.09190919 0.06935772]
 [0.08243562 0.13287465 0.08844408 0.09312289 0.09695429 0.09601729
  0.11815221 0.10172717 0.10636923 0.08390252]
 [0.09005146 0.12366708 0.09985453 0.10473341 0.08919542 0.09665906
  0.08862505 0.10163439 0.10771602 0.09786353]
 [0.12250438 0.05948806 0.09960631 0.11687227 0.09835437 0.10778171
  0.1221358  0.0842126  0.11536675 0.07367778]
 [0.13201407 0.0725403  0.14135708 0.08838557 0.10089175 0.10910852
  0.10653408 0.07653113 0.0816934  0.09094413]
 [0.10053081 0.07815444 0.0945551  0.11850528 0.09333014 0.0925977
  0.12322283 0.09560651 0.11962572 0.08387151]
 [0.10574647 0.096359   0.06850836 0.1208141  0.1049513  0.09880988
  0.10280696 0.11042961 0.09969647 0.09187786]
 [0.1169

INFO:tensorflow:loss = 2.0976536, step = 801 (16.360 sec)
INFO:tensorflow:probabilities = [[0.10324285 0.08910906 0.1056846  0.08493397 0.12059931 0.1092264
  0.0928141  0.09876735 0.10455026 0.09107219]
 [0.12797035 0.11253294 0.08121041 0.10968816 0.09081876 0.10802161
  0.081917   0.09896436 0.11885405 0.07002232]
 [0.12295457 0.08936337 0.10539308 0.10008848 0.08775065 0.09614547
  0.1026315  0.09706707 0.11597759 0.08262815]
 [0.1466241  0.06902249 0.12608875 0.09841716 0.08977595 0.11950863
  0.10277259 0.08206726 0.10011902 0.06560396]
 [0.08130457 0.1336792  0.07400324 0.0975469  0.08147144 0.09306293
  0.13280734 0.10287763 0.1110974  0.09214933]
 [0.06757624 0.12137686 0.08539523 0.11883066 0.09266821 0.09183896
  0.09521887 0.09650415 0.12730886 0.10328199]
 [0.08904239 0.1189275  0.11263238 0.08655435 0.07512012 0.0919623
  0.10356581 0.11576405 0.11904398 0.08738713]
 [0.11128964 0.07654408 0.13801935 0.09178347 0.08938227 0.09278075
  0.11326659 0.07553744 0.13662948 0.07

INFO:tensorflow:global_step/sec: 6.42245
INFO:tensorflow:probabilities = [[0.14146096 0.11126386 0.06008677 0.09647709 0.0913455  0.10517608
  0.09255926 0.0851538  0.12148448 0.09499224]
 [0.1099463  0.06060928 0.09754742 0.08416446 0.1167527  0.10903374
  0.11934169 0.09960659 0.10151619 0.10148156]
 [0.15083916 0.05489314 0.11279427 0.08741762 0.09294004 0.07948334
  0.11391369 0.11711364 0.14029    0.05031498]
 [0.12103273 0.09058598 0.09303971 0.09845752 0.0854217  0.08869847
  0.10451605 0.09439536 0.1338553  0.08999706]
 [0.09983473 0.09641843 0.09202768 0.10278115 0.09113464 0.09815087
  0.12883396 0.094695   0.1087115  0.08741205]
 [0.08631627 0.09789427 0.08803692 0.11725444 0.08119149 0.10992499
  0.10776757 0.08245825 0.12137628 0.10777957]
 [0.11344595 0.0865223  0.11297632 0.11188345 0.09443712 0.09837792
  0.098841   0.07711775 0.13767615 0.06872201]
 [0.06436616 0.1067006  0.1056212  0.10398863 0.07358872 0.10830369
  0.1261704  0.0869003  0.12772413 0.0966362 ]
 [0.107

INFO:tensorflow:loss = 2.0410523, step = 901 (15.565 sec)
INFO:tensorflow:probabilities = [[0.11235426 0.06323261 0.11384963 0.09453367 0.06999036 0.11954957
  0.08635321 0.09722067 0.14109659 0.10181949]
 [0.10671057 0.08957671 0.06871739 0.10815205 0.10686554 0.08707019
  0.07487524 0.11060458 0.12398912 0.12343857]
 [0.10112478 0.08343951 0.07416353 0.09326526 0.10743222 0.09710442
  0.09599523 0.10945752 0.12607007 0.11194752]
 [0.07348866 0.09193307 0.06704893 0.1388205  0.103443   0.09084942
  0.08051703 0.114973   0.09506875 0.14385757]
 [0.09446349 0.07685295 0.12217861 0.11655788 0.06865284 0.08271676
  0.13752511 0.11349265 0.11464418 0.07291556]
 [0.07441778 0.1122008  0.06999795 0.088477   0.11498914 0.11364808
  0.11163413 0.08632627 0.10089419 0.1274147 ]
 [0.07279626 0.12145329 0.09305141 0.09872891 0.09525114 0.09865569
  0.09754537 0.11350246 0.11660916 0.09240627]
 [0.09442423 0.13732632 0.09340467 0.09899059 0.0775589  0.09448702
  0.0927717  0.10427711 0.13123062 0.

INFO:tensorflow:Saving checkpoints for 1000 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 1.9900362.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-24-01:22:39
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-24-01:22:43
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.6852, global_step = 1000, loss = 1.9816396
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/mnist_convnet_model/model.ckpt-1000
{'accuracy': 0.6852, 'loss': 1.9816396, 'global_step': 1000}
